# Walkthrough for nuskybgd-py

Updated: 2021/09/21
By: BG (bwgref@srl.caltech.edu)

### Pre-requisites

1. Install HEASoft
2. Install pyXspec
3. Install nuskybgd-py
4. Download your data
5. Process observation using ```nupipeline``` (standard processing through Stage 2)
6. For simplicity, work in the SEQID/event_cl subdirectory. 
7. Make an event_cl/bgd directory

Note that the example here is for N132D, a dataset that I've used ```nuskybgd``` on fairly extensively. This is sequence ID 10601407002.

# Define source and background background regions

1. Make an image using nuproducts and/or using ds9 over the 3--20 keV bandpass



In [ ]:
from importlib import reload
import nustar_gen as nsg
from nustar_gen import info
from nustar_gen import wrappers
reload(wrappers)
import os
here = os.getcwd()+'/'

import subprocess
from subprocess import DEVNULL, STDOUT


# Required to run heasoft stuff in the background
%env HEADASNOQUERY=
%env HEADASPROMPT=/dev/null


obs = info.Observation(path=here, seqid='10601407002')
obs.evtfiles['A'][0]
obs.exposure_report()
print(obs.observation_date)

In [ ]:
# Make a 3--20 keV image and a 20--40 keV image for both FPMs
print(f'Output images are produced here here: {obs.out_path}')

for mod in ['A', 'B']: 
    wrappers.make_image(infile=obs.evtfiles[mod][0], elow=3, ehigh=20)
    wrappers.make_image(infile=obs.evtfiles[mod][0], elow=20, ehigh = 40)
    
# Set up an outdir
outdir = os.path.join(obs.evdir, 'bgd')
print(outdir)
# Make the directory if it doesn't already exist
try:
    os.path.isdir(outdir)
except:
    print()


# Define background regions, cont'd


1. Try to generate regions *per detector*. Otherwise you are mixing instrument responses (and that can be bad or hard to diagnose later). Put these region files in the ```outdir``` that you specified above.

![](./bgd_regions.pdf)


2. Avoid any stray light regions when defining your backgrounds.
4. As a sanity check, if you are near any bright X-ray binaries, make a 20--40 keV image as well and look for any signs of transmitted stray light (I did not do that here)
5. Save the *individual* region files (not everything in one) into the bgd directory. Here I called my regions ```bgdA0.reg```, ```bgdA1.reg```, and ```bgdA3.reg``` since they were on detectors 0, 1 and 3. Do the same for FPMB.
6. Use nuproducts to generate a spectrum for each of these background regions
    * Note: The easiest way to do this is to use the “make_spectrum” wrapper in nustar-gen-utils, illustrated below.
7. You'll probably want a full ```nuproducts``` run for the source spectrum. The ```wrapper.make_spectra``` below will work fine (just turn the runmkarf= parameter on), but editing your own script for extended source / etc, lightcurves is up to the user.

# Generate a spectrum for each of your background regions

In [ ]:
# Uncomment the subprocess.run line bel to run this directly in the notebook
# This can take a little while, so go run these shell scripts in the terminal

bgd_regions = {'A':[0, 1, 3], 'B':[0, 1, 2, 3]}
for mod in bgd_regions:
    ev = obs.evtfiles[mod][0]
    for reg in bgd_regions[mod]:
        reg_file = os.path.join(outdir, f'bgd{reg}{mod}.reg')        
        outscr = nsg.wrappers.make_spectra(ev, mod,reg_file,
                                  runmkarf='no',
                                  outpath=outdir)
        print(outscr)
        


# You could have just followed the nuskybgd readme file, but it uses some other calls that aren't as flexible.

For now, go to Step 3 in [the nuskybgd readme file](https://github.com/achronal/nuskybgd-py). I'll reproduce that here:

1. Go to the event_cl directory in a terminal window
2. Initialize the nuskybgd setup
3. Make the instrument maps

```
nuskybgd mkinstrmap nu10601407002A01_cl.evt

nuskybgd mkinstrmap nu10601407002B01_cl.evt
```
4. Make the aspect histograms

```
nuskybgd aspecthist nu10601407002A_det1.fits gtifile=nu10601407002A01_gti.fits \
    out=aspecthistA.fits

nuskybgd aspecthist nu10601407002B_det1.fits gtifile=nu10601407002B01_gti.fits \
    out=aspecthistB.fits
```

5. Go to event_cl/bgd
6. Generate the background aperture images. Change the refimg parameter if you called the image file something different.

```
nuskybgd projbgd refimg=../nu10601407002A01_cl_3to20keV.fits out=bgdapA.fits \
	mod=A det=1234 chipmap=../newinstrmapA.fits aspect=../aspecthistA.fits

nuskybgd projbgd refimg=../nu10601407002A01_cl_3to20keV.fits out=bgdapB.fits \
	mod=B det=1234 chipmap=../newinstrmapB.fits aspect=../aspecthistB.fits
```

# Go to the bgd directory and produce an input json file. Call it something like bgdinfo.json.

It should look something like this, where you're going to use the various inputs. Note that I've found that we're typically better off just fitting the two telescopes independently first and *then* combining in a final ```XSPEC``` run. 

### Important values to set:

-     "intbgd_fix_line_ratios": true

This is important for tying together all of the internal line components. For *most* part this is what you actually want to do. This is especially true if you have observations <100-ks where the individual background lines are going to be poorly constrained.

- fcxb_config

This line ties together the focused CXB component. This is somewhat esoteric, but you want the focused CXB to be the same *at the different locations in the FoV* not **for the same detector**. In practice, I think we will typically keep this fixed.

- Make sure that the regfiles and bgfiles line up

This is an obvious point, but this tells nuskybgd which version to use.


---
```
{
    "bgfiles": [
         "nu10601407002A01_bgd0A_sr.pha",
         "nu10601407002A01_bgd1A_sr.pha",
         "nu10601407002A01_bgd3A_sr.pha"
    ],

    "regfiles": [
        "bgd0A.reg",
        "bgd1A.reg",
        "bgd3A.reg"
    ],

    "refimgf": "bgdapA.fits",

    "bgdapfiles": {
        "A": "bgdapA.fits",
    },
    "bgddetfiles": {
        "A": [
            "det0Aim.fits",
            "det1Aim.fits",
            "det2Aim.fits",
            "det3Aim.fits"
        ]
    },
    
    "intbgd_fix_line_ratios": true
}

```

# Set up .xcm file for fitting like this:

```
nuskybgd fit fpma.json savefile=n132d_2020_fpma >& fita.log
```

## Some notes

- This will produce a .xcm file called n132d_2020_fpma.xcm, which will be *starting* point for Xspec fitting of the background.
- If you change the json and rerun it again with teh same ```savefile=``` parameter then it will silently append a numeric before the .xcm extension. So just beware that you are using the correct file.


# Start of Xspec Fitting

## Start a wrapper .xcm file to use.
- Note that you will probably *not* run this via @script.xcm but just copy and paste things as you go into the ```Xspec>``` prompt

### Rescale the internal norms correctly

```
@n132d_2020_fpma.xcm
cpd /xw
pl ld rat

# Rescale intn norms
newpar intn:4 1e-4
statistic cstat
```

- ...for some reason the internal background continuum is off by 1e4. Not sure what's wrong in the model, but this is never correct. I always scale it to 1e-4.

- Use cstat. If you want to, you can use the ftgrouppha to "optimally" bin the data first. At this stage I don't think it makes a ton of difference. Because this is a phenomenological model you don't expect to *ever* get a fit statistic comparable to the degrees of freedom. But if it makes you feel better, defintely go for it.



### Turn off the "solar" apec component and associated lines


```
newpar intbgd:4 0. -0.1
newpar intbgd:10 0. -0.1
newpar intbgd:7 0. -0.1
newpar intbgd:13 0. -0.1

```

- ...all of these "background" spectral components were introduced because the COSMOS background fields occured when there were lots of solar flares. These are *not* generally required and we generally recommend filtering these out by hand using GTIs first rather than trying to fit them. They have a different spatial distribution than all of the other background components. If you want to check for the presence of solar activity, see **(some jupyter notebook I haven't written yet)**.

- Note that unfortunately the code does not allow you to simply remove these model components, so we just zero them out and freeze them.

### Freeze the focused CXB

```
freeze fcxb:*
```

- ...this has to be there, but is usually difficult to constrain. Keep it fixed unless you have a good reason to allow it to vary.

### Check the "default" fit

It should looke something like this:

![](xspec_default.png)



### Do a preliminary fit and check the results

```
fit 1000
pl ld rat

```

...which should now have fit the continuum and CXB values:

![](xspec_preliminary.png)





## Make adjustments to the fit

In this case, the 88 keV line and the 24 keV line are different than the default values with resepect to the other lines. The 24 keV line we know appears after the SAA and the 88 keV line varies with time. You can go thaw these components in the background model as necessary.

Here it's important to note that the strength of some of these backgroud components varies *per detector* in ways that are not captured by the ```nuskybgd``` model. Which is why you're generally better off fitting on a "per detector" basis.

Once you're happy, save your fit in something like this:

```
save all n132d_2020_fpma_fit.xcm
```

## Produce the "simultaneous" .xcm file

```
nuskybgd spec fpma.json n132d_2020_fpma_fit.xcm srcA.reg nu10601407002A01_srcA_sr.pha
```

- Note: there is syntax for doing both FPMs at the same time, but I think you're typically going to be better off doing this epoch-by-epoch and FPM -by-FPM.

- This produces a .xcm file usually bgdA.xcm or bgdB.xcm. If you run things again, then it will make something like bgdA2.xcm. This file is echoed to the command line, so keep that in mind when you run this.

## Check the projection

bgdA.xcm now has the background and source spectrum all loaded in. In principle, you can use this data and simultaneously fit both the background model in both the source *and* background regions (which is statistically correct). However, this tends to be computationally expensive and does not allow for any variations in the underlying spatial model (or relative strength between detectors).

In my case, it looks like this, where I have combined the two background regions (the black curve) and am showing the source counts spectrum.


![](xspec_source_and_bgd.png)



## Finally, "simplify" the xcm file

If you want to just retain a (fixed) background model projected into your source region, you can use the following syntax:

```
nuskybgd simplify fpma.json bgd_srcA.xcm

```
...which produces "bgd_only_srcA.xcm", which is *actually* just the source spectrum with its background model. When you load this is to Xspec via @bgd_only_srcA.xcm you 

...so that when you load the bdg_srcA.xcm file it now contains *only* the projected background spectrum. You can still add a source model using the

```
model powerlaw
```

...or whatever since "model 1" is reserved by ```nuskybgd``` for the actual source model.

At this point it's also possible to merge things together (by hand) so that you can fit multiple epochs and/or both FPMs. **We can give an example here?** This is not too complicated to do by hand for a limited number of observations, and could be semi-automated in the future. If you do this, then it is *not* recommend to try to simultaneously fit the background.

# Final notes

- It's left up to the user to figure out what to do next as far as using the "correct" fit energy range and models. I'd typically recommend here using ftgrouppha and limiting yourself to only fitting up until the background starts to become dominant. Otherwise you're just including the extra degrees of freedom from the background fit, which will not be correct.

- It's definitely not clear that any fit statistics will be chi-square distributed. So do your own dilligence to make sure you understand goodness of fit.